In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://world.taobao.com/search/json.htm?navigator=all&_ksTS=1481852895654_41&spm=a21bp.7806943.20151106.1&search_type=0&_input_charset=utf-8&json=on&q=100%25%E5%85%A8%E9%BA%A6%E7%B2%89&cna=Dd0TEMtX5wsCAXPBrNqwX4B1&callback=__jsonp_cb&abtest=_AB-LR517-LR854-LR895-PR517-PR854-PR895&nid=&type=&uniqpid='

wb_data = requests.get(url)

wb_html = wb_data.text

In [3]:
soup=BeautifulSoup(wb_html,'lxml')
soup.select('div > div > ul > li')  # 找不到。因为数据独立存于js里，并不能一般地解析DOM结构

[]

## 用Json 和正则

In [4]:
import json
import re

In [5]:
#wb_data.text # 或者print(soup.text)

In [6]:
m = re.search(r'if\(window.__jsonp_cb\)\{__jsonp_cb\((.*?)\)\}',wb_data.text) 
                                    # 因为无法确定中间是否有其他重复内容导致re.search不完整，
                                    # 索性通过开头结尾的唯一性来获取完整的全部内容，通过小括号包裹想要的内容
#print(m.group(1))                   # 再用group(1)。从而保证完整性

In [7]:
jd = json.loads(m.group(1))   # 将Json 字符串 转换成 python数据结构

## 若是简单的对齐的数据，可以直接将字典转换成pandas.DataFrame，但发现这个数据结构很复杂，字典里套字典。如何观察提炼？

In [8]:
# 方法一：将Json字符串（不是字典）写入文档再通过浏览器读取，同样的检查元素操作，preview会重新排版，方便观察数据；然后用python字典的方式提炼
# 方法二：同样通过在原网页结构中，检查各元素定位属性 如 'div id=list-itemList' 但相对繁琐

with open('taobao.json','w') as f:  
    f.write(json.dumps(jd))   #  然后用浏览器打开这个json文件

In [9]:
for item in jd['itemList']: # 观察发现，商品信息位于‘itemList’
    print(item['nick'],'\t',item['tip'],'\t',item['price'])     # 店铺名称，价格
    

沂蒙山村杂粮铺 	 全麦面粉 农家现磨石磨含麦麸皮小麦粉 100%全麦粉面包粉 包邮5斤 	 27.00
ku酷衣 	 张家口坝上张北100%纯莜面面粉莜麦面粉燕麦面粉全麦莜面粉8斤 	 28.00
yan_boy 	 【弘毅生态农场】六不用全麦面粉非转基因（10斤）出粉率100% 	 85.00
liubohs9201180 	 全麦粉 高筋面粉 含麦麸的100% 烘培原料 无添加剂 5kg袋装 包邮 	 51.77
ku酷衣 	 张家口坝上张北100%纯莜面面粉莜麦面粉燕麦面粉全麦莜面粉5斤包 	 31.80
栗子的一切 	 【亦姜妈妈放心农产品】2015全麦面粉石磨面粉100%麸皮带胚芽 	 5.50
周月平8888 	 100%纯全麦荞面粉 荞麦面粉 荞面无糖食品面粉荞麦面1000克 	 10.00
食里香农家店 	 农家自种100%纯生黑麦粉 裸麦粉 麦麸全麦面粉杂粮面包粉五斤包邮 	 6.50
蔡武不卖假货 	 日本制粉Nippn100%全麦快熟营养扭扭通心粉(2分钟) 日期最新 	 10.20
蔡武不卖假货 	 日本Nippn100%全麦多彩4分钟快熟营养通心粉扭扭通粉150g日期最新 	 10.00
卡米的幻想 	 日本制粉Nippn100%全麦快熟营养扭扭粉 通心粉通心面 2分钟 200g 	 28.00
fade_bogod 	 日本Nippn 100%全麦婴儿童宝宝辅食快熟2分钟营养面条通心粉 200g 	 22.00
阳的快乐生活 	 日本制粉Nippn100%全麦多彩快熟营养通心粉/扭扭通粉(4分钟快熟) 	 16.00
卡车族 	 日本Nippn儿童100%全麦快熟营养扭扭通心粉 2分钟宝宝意面条 200g 	 20.00
young_ting 	 日本制Nippn100%全麦谷物通粉幼儿童宝宝扭扭面条2分钟即食手动 	 13.50
春之青叶 	 日本制粉Nippn100%全麦快熟营养扭扭芬 通心粉通心面(2分钟)200g 	 15.80
ljf580231 	 张家口张北100%纯莜面莜麦面粉燕麦面粉全麦莜面粉笨莜面8斤 	 38.00
youyoucaoxilie 	 100%纯全麦荞面粉 荞麦面粉 荞面饸饹面条无糖食品面粉 8斤装包邮 	 35.00
meierxue88 	 山西大同特产无糖100%纯莜面面粉莜麦燕麦面粉全麦莜面粉

In [10]:
# 也可以直接用pandas.DataFrame。
df=pd.DataFrame(jd['itemList'])
#df.head()

# 数据规整：删除含有缺失值的列;将一些数值列dtypes由object 转换为数值型 

In [11]:
import numpy as np

df=df.applymap(lambda x: np.nan if x =='' else x)  # 此处用匿名函数和广播方法，填充缺失值

In [12]:
#df.isnull().sum()

In [13]:
df=df.dropna(axis=1)  # 删除含有缺失值的列

In [14]:
df=df.apply(lambda x:pd.to_numeric(x,errors='ignore')) # 转换dtypes